<a href="https://www.kaggle.com/mickaelnarboni/clients-segmentation-rfm-maintenance?scriptVersionId=88676724" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Table of Contents

* [Dataframes frequency](#dataframes-frequency)
* [Clusters stability](#clusters-stability)
* [Adjusted Rand Index](#ari)
* [Final thoughts](#final-thoughts)


Import the relevant library for the notebook.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore") # ignore the warnings about file size
import matplotlib.pyplot as plt
from matplotlib import colors
%matplotlib inline
import seaborn as sns
from time import process_time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import math
from math import *
from sklearn.metrics import adjusted_rand_score
import plotly.io as pio
pio.renderers.default = 'iframe'
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.cluster import Birch
from sklearn.cluster import SpectralClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import MiniBatchKMeans
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from yellowbrick.contrib.scatter import ScatterVisualizer
import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.express as px


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/olist-clients-segmentation/database_p4.csv
/kaggle/input/olist-clients-segmentation/geolocation_p4.csv
/kaggle/input/olist-clients-segmentation/rfm_clustering.csv


Import our dataframe. 

In [2]:
df = pd.read_csv('../input/olist-clients-segmentation/rfm_clustering.csv',sep='\t', index_col=[0], low_memory=False)
df

,CustomerUniqueID,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
0,861eff4711a542e4b93843c6dd7febb0,474,1,1,0,124.99,0,1.078731,-0.574149,0.163933,3,Gold
1,290c77bc529b7ac935b93aa66c333dc3,233,0,1,0,289.00,0,0.283691,-0.574149,0.981946,3,Gold
2,060e732b5b29e8181a18229c7b0b2b5e,106,3,1,0,139.94,0,-0.598033,-0.574149,0.274193,3,Gold
3,259dac757896d24d7702b9acbbff3f3c,173,0,1,0,149.94,0,-0.049639,-0.574149,0.341553,3,Gold
4,345ecd01c38d18a9036ed96c73b8d066,35,3,1,0,230.00,0,-1.838550,-0.574149,0.759096,2,Silver
...,...,...,...,...,...,...,...,...,...,...,...,...
115604,1a29b476fee25c95fbafc67c5ac95cf8,148,0,1,0,74.90,0,-0.224371,-0.574149,-0.335818,0,Bronze
115605,d52a67c98be1cf6a5c84435bd38d095d,152,0,1,0,114.90,0,-0.194516,-0.574149,0.081788,3,Gold
115606,e9f50caf99f032f0bf3c55141f019d99,147,0,1,0,37.00,0,-0.231961,-0.574149,-1.024074,0,Bronze
115607,73c2643a0a458b49f58cea58833b192e,303,2,1,0,689.00,3,0.577779,-0.574149,1.829843,3,Gold


Return the shape of the dataframe. 

In [3]:
df.shape

(115609, 12)

We order the entries by Recency so it'll be easier for us to create regular interval of recency to split our customers.
We notice that our latest customer order has been fulfilled 728 days ago (almost 2 years) before downloading the Olist data.

In [4]:
df.sort_values(by=['Recency'], ascending=False, inplace=True)
df

,CustomerUniqueID,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
99531,b7d76e111c89f7ebf14761390f0f7d17,728,1,2,0,72.89,0,1.559105,0.625897,-0.362365,0,Bronze
99530,b7d76e111c89f7ebf14761390f0f7d17,728,1,2,0,72.89,0,1.559105,0.625897,-0.362365,0,Bronze
83867,4854e9b3feff728c13ee5fc7d1547e92,728,1,1,0,59.50,0,1.559105,-0.574149,-0.560453,0,Bronze
105686,0eb1ee9dba87f5b36b4613a65074337c,700,1,1,0,100.00,0,1.515197,-0.574149,-0.053760,3,Gold
9726,61db744d2f835035a5625b59350c6b63,699,1,1,0,36.49,0,1.513596,-0.574149,-1.037620,0,Bronze
...,...,...,...,...,...,...,...,...,...,...,...,...
37910,b4dcade04bc548b7e3b0243c801f8c26,4,3,1,0,99.00,0,-4.266824,-0.574149,-0.063568,2,Silver
74708,7a22d14aa3c3599238509ddca4b93b01,4,3,1,0,63.90,0,-4.266824,-0.574149,-0.490828,2,Silver
60274,b701bebbdf478f5500348f03aff62121,4,3,1,0,24.90,0,-4.266824,-0.574149,-1.410589,2,Silver
49667,ff22e30958c13ffe219db7d711e8f564,0,3,2,0,290.00,0,-5.818792,0.625897,0.985317,2,Silver


<a id="dataframes-frequency"></a>
## Dataframes frequency

We split the dataframe into dataframes of 10,000 orders based from the latest to the newest recency of clients to know until what point the model will be obsolete and need a maintenance work by Olist. 

In [5]:
B0 = df.iloc[0:10000]
B1 = df.iloc[10000:20000]
B2 = df.iloc[20000:30000]
B3 = df.iloc[30000:40000]
B4 = df.iloc[40000:50000]
B5 = df.iloc[50000:60000]
B6 = df.iloc[60000:70000]
B7 = df.iloc[70000:80000]
B8 = df.iloc[80000:90000]
B9 = df.iloc[90000:100000]
B10 = df.iloc[100000:110000]

We create a dataframe for each subdataframe that will contain our three RFM variables after transformation. 

In [6]:
rfm_B0 = B0[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B1 = B1[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B2 = B2[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B3 = B3[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B4 = B4[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B5 = B5[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B6 = B6[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B7 = B7[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B8 = B8[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B9 = B9[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B10 = B10[['Recency_standard','Frequency_standard','Monetary_value_standard']]

<a id="clusters-stability"></a>
## Clusters stability

Create a KMeans method with the same hyperparameters tested in the modeling notebook.

In [7]:
# Use the hyperparameters from our model K=4 using KMeans clustering method
kmeans_cluster = KMeans(n_clusters=4)

Create a model M0 to fit the subdataframe B0.

In [8]:
M0 = kmeans_cluster.fit(rfm_B0)

Create a model M1 to fit the subdataframe B1.

In [9]:
M1 = kmeans_cluster.fit(rfm_B1)

Create the array of labels of our model M0 to predict the subdataframe B0.

In [10]:
label_00 = M0.predict(rfm_B0)

Create the array of labels of our model M1 to predict the subdataframe B0.

In [11]:
label_01 = M1.predict(rfm_B0)

Create the ARI between the predictions of M0 and M1 on the subdataframe B0.

In [12]:
ariB01 = adjusted_rand_score(label_00, label_01)
ariB01

1.0

Create a model M2 to fit the subdataframe B2.

In [13]:
M2 = kmeans_cluster.fit(rfm_B2)

Create the array of labels of our model M2 to predict the subdataframe B0.

In [14]:
label_02 = M2.predict(rfm_B0)

Create the ARI between the predictions of M1 and M2 on the subdataframe B0.

In [15]:
ariB02 = adjusted_rand_score(label_01, label_02)
ariB02

0.8909690868183856

Create a model M3 to fit the subdataframe B3.

In [16]:
M3 = kmeans_cluster.fit(rfm_B3)

Create the array of labels of our model M3 to predict the subdataframe B0.

In [17]:
label_03 = M3.predict(rfm_B0)

Create the ARI between the predictions of M2 and M3 on the subdataframe B0.

In [18]:
ariB03 = adjusted_rand_score(label_02, label_03)
ariB03

0.9100119024159246

Create a model M4 to fit the subdataframe B4.

In [19]:
M4 = kmeans_cluster.fit(rfm_B4)

Create the array of labels of our model M4 to predict the subdataframe B0.

In [20]:
label_04 = M4.predict(rfm_B0)

Create the ARI between the predictions of M3 and M4 on the subdataframe B0.

In [21]:
ariB04 = adjusted_rand_score(label_03, label_04)
ariB04

0.9369127285408371

Create a model M5 to fit the subdataframe B5.

In [22]:
M5 = kmeans_cluster.fit(rfm_B5)

Create the array of labels of our model M5 to predict the subdataframe B0.

In [23]:
label_05 = M5.predict(rfm_B0)

Create the ARI between the predictions of M4 and M5 on the subdataframe B0.

In [24]:
ariB05 = adjusted_rand_score(label_04, label_05)
ariB05

0.9685294221557417

Create a model M6 to fit the subdataframe B6.

In [25]:
M6 = kmeans_cluster.fit(rfm_B6)

Create the array of labels of our model M6 to predict the subdataframe B0.

In [26]:
label_06 = M6.predict(rfm_B0)

Create the ARI between the predictions of M5 and M6 on the subdataframe B0.

In [27]:
ariB06 = adjusted_rand_score(label_05, label_06)
ariB06

0.9703370381664267

Create a model M7 to fit the subdataframe B7.

In [28]:
M7 = kmeans_cluster.fit(rfm_B7)

Create the array of labels of our model M7 to predict the subdataframe B0.

In [29]:
label_07 = M7.predict(rfm_B0)

Create the ARI between the predictions of M6 and M7 on the subdataframe B0.

In [30]:
ariB07 = adjusted_rand_score(label_06, label_07)
ariB07

0.9343513481864125

Create a model M8 to fit the subdataframe B8.

In [31]:
M8 = kmeans_cluster.fit(rfm_B8)

Create the array of labels of our model M8 to predict the subdataframe B0.

In [32]:
label_08 = M8.predict(rfm_B0)

Create the ARI between the predictions of M7 and M8 on the subdataframe B0.

In [33]:
ariB08 = adjusted_rand_score(label_07, label_08)
ariB08

0.9328303510961014

Create a model M9 to fit the subdataframe B9.

In [34]:
M9 = kmeans_cluster.fit(rfm_B9)

Create the array of labels of our model M9 to predict the subdataframe B0.

In [35]:
label_09 = M9.predict(rfm_B0)

Create the ARI between the predictions of M8 and M9 on the subdataframe B0.

In [36]:
ariB09 = adjusted_rand_score(label_08, label_09)
ariB09

0.9907042923875182

Create a model M10 to fit the subdataframe B10.

In [37]:
M10 = kmeans_cluster.fit(rfm_B10)

Create the array of labels of our model M10 to predict the subdataframe B0.

In [38]:
label_10 = M10.predict(rfm_B0)

Create the ARI between the predictions of M9 and M10 on the subdataframe B0.

In [39]:
ariB10 = adjusted_rand_score(label_09, label_10)
ariB10

0.9084263348155587

<a id="ari"></a>
## Adjusted Rand Index

The ARI allows us to score our model stability to see if the predicted clustering are sorted in the same clusters as our model. The closer the score is from 1, the better.
We do that on the three variables we've been building in our model notebook: Recency, Frequency, Monetary Value.
We apply the models to each new dataframe we previously created to track the evolution of the stability along time.

We create a table that contains all our ARI scores. 

In [40]:
d = {'ARI'}
list1 = [ariB01, ariB02, ariB03, ariB04, ariB05, ariB06, ariB07, ariB08, ariB09, ariB10]
ari_table = pd.DataFrame(data = list1, columns=d)
ari_table

,ARI
0,1.000000
1,0.890969
2,0.910012
3,0.936913
4,0.968529
5,0.970337
6,0.934351
7,0.932830
8,0.990704
9,0.908426


Plot a curve of the ARI on each subdataframe.
We get the ARI for each new 10,000 orders based on recency from the latest to the newest.

In [41]:
pd.options.plotting.backend = "plotly"
fig = ari_table.plot(title = "RFM Predictions for clients segmentation with KMeans using ARI", labels=dict(index="10k Orders Intervals", value="Precision Score", variable=""))
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        ticktext = ['10k', '20k', '30k', '40k', '50k', '60k', '70k', '80k', '90k', '100k']
    )
)
fig.show()

<a id="final-thoughts"></a>
## Final thoughts

Our model stays correct overtime, it doesn't require a maintenance in the first **two years of orders**. 